# implement LRA proposed by  Wu, Jianshe et al. "Computing the Number of Loop-Free K-Hop Paths of Networks",IEEE Transactions on Services Computing 15.4 (2020): 2114-2128.

In [ ]:

import torch

def lra_2_hop(A):
    P2 = A @ A
    P2.fill_diagonal_(0)
    return P2

def lra_3_hop(A):
    degrees = A.sum(dim=1).unsqueeze(0)
    P2 = lra_2_hop(A)
    Q3 = P2 @ A
    P3 = Q3 - A * (degrees - 1)
    P3.fill_diagonal_(0)
    return P3

def lra_4_hop(A):
    P3 = lra_3_hop(A)
    P2 = lra_2_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    degrees = A.sum(dim=1).unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)

    P4_0 = Q4 - (degrees - 1) * P2
    P4_1 = Q4 - (degrees - 2) * P2 - (Q3_diag - 2 * P2)

    P4 = P4_0 * (1 - A) + P4_1 * A
    P4.fill_diagonal_(0)
    return P4

def lra_5_hop(A):
    P2 = lra_2_hop(A)
    P3 = lra_3_hop(A)
    P4 = lra_4_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    Q5 = P4 @ A

    degrees = A.sum(dim=1).unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)
    Q4_diag = Q4.diagonal().unsqueeze(0)

    P5_0 = Q5.clone()
    P5_0 -= (degrees - 1) * P3 - A @ (A * P2)
    P5_0 -= (A @ A) * Q3_diag - 2 * A @ (A * P2)

    P5_1 = Q5.clone()
    P5_1 -= (degrees - 1) + (degrees - 2) * P3 - A @ (A * P2)
    P5_1 -= (A @ A) * (Q3_diag - 2 * P2 + 2) - 2 * A @ (A * P2)
    P5_1 -= Q4_diag - 2 * P3 - P2 * (P2 - 1)

    P5 = P5_0 * (1 - A) + P5_1 * A
    P5.fill_diagonal_(0)
    return P5

def lra_k_hop(A, k):
    if k == 1:
        return A
    elif k == 2:
        return lra_2_hop(A)
    elif k == 3:
        return lra_3_hop(A)
    elif k == 4:
        return lra_4_hop(A)
    elif k == 5:
        return lra_5_hop(A)

    P2 = lra_2_hop(A)
    P3 = lra_3_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    Q4_diag = Q4.diagonal().unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)

    degrees = A.sum(dim=1).unsqueeze(0)

    Pk = lra_k_hop(A, k - 1) @ A
    Pk -= (degrees - 1) + (degrees - 2) * P3 - A @ (A * P2)
    Pk -= (A @ A) * (Q3_diag - 2 * P2 + 2) - 2 * A @ (A * P2)
    Pk -= Q4_diag - 2 * P3 - P2 * (P2 - 1)
    Pk.fill_diagonal_(0)
    return Pk

# P(0)(I)) to P(7) computed using the LRA algorithm
def generate_q_matrices(A):
    A = A.to(torch.float64)  
    n = A.size(0)
    Q_matrices = []
    I = torch.eye(A.size(0), dtype=torch.float64) 
    Q_matrices.append(I)

    for k in range(1, 8):
        Qk = lra_k_hop(A, k)
        Q_matrices.append(Qk)

    return Q_matrices

# Implement the Revised LRA, as described in the Preface of this paper.

In [ ]:
'''
The corrected code correctly calculates the number of loop - free paths of order 5.
'''


import torch
def lra_5_hop_revised(A):
    P2 = lra_2_hop(A)
    P3 = lra_3_hop(A)
    P4 = lra_4_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    Q5 = P4 @ A

    degrees = A.sum(dim=1).unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)
    Q4_diag = Q4.diagonal().unsqueeze(0)

    P5_0 = Q5.clone()
    P5_0 -= (degrees - 1) * P3 - A @ (A * P2)
    P5_0 -= (A @ A) * Q3_diag - 2 * A @ (A * P2)

    P5_1 = Q5.clone()
    P5_1 -= (degrees - 2) * P3 - A @ (A * (P2-1))
    P5_1 -= (A @ A) * (Q3_diag - 2 * P2 + 2) - 2 * A @ (A * P2)
    P5_1 -= Q4_diag - 2 * P3 - P2 * (P2 - 1)

    P5 = P5_0 * (1 - A) + P5_1 * A
    P5.fill_diagonal_(0)
    return P5

'''
Since the computational expressions of length 5 have been modified, and the expressions with k > 5 require the results obtained when k = 5, they thus also need to be modified.
'''
def lra_k_hop_revised(A, k):
    if k == 1:
        return A
    elif k == 2:
        return lra_2_hop(A)
    elif k == 3:
        return lra_3_hop(A)
    elif k == 4:
        return lra_4_hop(A)
    elif k == 5:
        return lra_5_hop_revised(A)

    P2 = lra_2_hop(A)
    P3 = lra_3_hop(A)
    Q3 = P2 @ A
    Q4 = P3 @ A
    Q4_diag = Q4.diagonal().unsqueeze(0)
    Q3_diag = Q3.diagonal().unsqueeze(0)

    degrees = A.sum(dim=1).unsqueeze(0)

    Pk = lra_k_hop(A, k - 1) @ A
    Pk -= (degrees - 2) * P3 - A @ (A * (P2-1))
    Pk -= (A @ A) * (Q3_diag - 2 * P2 + 2) - 2 * A @ (A * P2)
    Pk -= Q4_diag - 2 * P3 - P2 * (P2 - 1)
    Pk.fill_diagonal_(0)
    return Pk
'''
Store the path count matrices from length 0 (i.e., the identity matrix I) to length 7, which are computed by Revised LRA, in a list.
'''
def generate_q_matrices_revised(A):
    A = A.to(torch.float64)
    n = A.size(0)
    Q_matrices = []
    I = torch.eye(A.size(0), dtype=torch.float64)
    Q_matrices.append(I)

    for k in range(1, 8):
        Qk = lra_k_hop_revised(A, k)
        Q_matrices.append(Qk)

    return Q_matrices

# Implement the diagonal matrix operation, that is, set all off-diagonal elements of the input matrix A to 0 while keeping the diagonal elements unchanged, and output the result.

In [3]:
import torch
def diag(matrix):
    """
    Input: An n×n PyTorch tensor (square matrix)
    Output: An n×n diagonal matrix with the same diagonal elements
    """
    # Extract the diagonal elements
    diag_elements = matrix.diagonal()
    
    # Create diagonal matrix using torch.diag()
    diag_matrix = torch.diag(diag_elements)
    
    return diag_matrix

# Implement the calculation of CNR according to Algorithm 1 in this paper.

In [4]:
# Compute the CNR
def common_neighbor_similarity_sparse_cpu(A):
    dense_common = A@A
    valid_pairs = torch.nonzero(dense_common, as_tuple=False)
    
    num_vertices = A.shape[0]
    similarity = torch.zeros(num_vertices, num_vertices, dtype=torch.float32)
    
    # Process all valid vertex pairs in batches
    for i, j in valid_pairs:
        i, j = i.item(), j.item()
        num_common = int(dense_common[i, j].item())
        
        if num_common < 2:
            continue
            
        # Get indices of common neighbors
        common_neighbor_indices = (A[i].to_dense() * A[j].to_dense()).nonzero(as_tuple=True)[0]
        
        # Calculate adjacent pairs using matrix operations
        neighbors_adj = A[common_neighbor_indices][:, common_neighbor_indices].to_dense()
        count_adjacent = neighbors_adj.sum().item()
        
        total_pairs = num_common * num_common
        similarity[i, j] = count_adjacent / total_pairs
    
    return similarity


# Define the operation of a diagonal matrix multiplied by a matrix

In [5]:
import torch

def diag_matmul_left(D_diag, M):
    """
    Efficiently compute the product of a diagonal matrix D and a regular matrix M
    
    Args:
    D_diag (torch.Tensor): Diagonal elements of diagonal matrix D, shape [n]
    M (torch.Tensor): Regular matrix, shape [n, m]
    
    Returns:
    torch.Tensor: Result matrix, shape [n, m]
    """
    diagonal = torch.diag(D_diag)           # Extract diagonal elements 
    column_vector = diagonal.unsqueeze(1)   # Convert to column vector (n, 1)
    return column_vector * M  # Equivalent to D @ M, but more efficient

def diag_matmul_right(M, D_diag):
    """
    Efficiently compute the product of a regular matrix M and a diagonal matrix D
    
    Args:
    M (torch.Tensor): Regular matrix, shape [m, n]
    D_diag (torch.Tensor): Diagonal elements of diagonal matrix D, shape [n]
    
    Returns:
    torch.Tensor: Result matrix, shape [m, n]
    """
    return M * torch.diag(D_diag)  # Equivalent to M @ D, but more efficient

# Implement the computational expression of P(6) proposed in this paper, where the detailed expression is contained in P(6) and P(7) Equation.pdf in the supplementary materials.

In [6]:
import torch

def compute_P6(A):
    A2 = torch.matrix_power(A, 2)
    A3 = torch.matrix_power(A, 3)
    A4 = torch.matrix_power(A, 4)
    A5 = torch.matrix_power(A, 5)
    A6 = torch.matrix_power(A, 6)
    D2 = diag(A2)
    D3 = diag(A3)
    D4 = diag(A4)
    D5 = diag(A5)
    D6 = diag(A6)
    AD2A = A@diag_matmul_left(D2,A)
    D22 = D2*D2
    A2D2 = diag_matmul_right(A2,D2)
    D3A = diag_matmul_left(D3,A)
    AD3 = diag_matmul_right(A,D3)
    P6 = (A6 - (
        diag_matmul_left(D2,A4) + diag_matmul_right(A4,D2)
        + diag_matmul_right(A,D2)@A3 + diag_matmul_right(A3,D2)@A
        + A2D2@A2 + AD3@A2
        + A2@D3A + diag_matmul_right(A,D4)@A + diag_matmul_left(D3,A3)
        + diag_matmul_right(A3,D3) + diag_matmul_right(A2,D4) + D4@A2
        + diag_matmul_right(A,D5) + diag_matmul_left(D5,A) + D6
    )+(
        4*A4 + diag_matmul_right(A2,diag(AD2A)) + diag_matmul_left(diag(AD2A),A2)
        + diag_matmul_left(D2,A2D2) + 2*diag_matmul_right(A,D22)@A
        + 2*diag_matmul_left(D22,A2) + 2*diag_matmul_right(A2,D22)
        + A@diag_matmul_left(diag(AD2A),A) + diag_matmul_right(AD2A,D2) + diag_matmul_left(D2,AD2A)
        + 4*diag_matmul_right(A,(D2*D3)) + 4*diag_matmul_left(D2,D3A) + 3*A@(A2*A)@A
        + 2*diag_matmul_right(A,diag(A2D2@A)) + 2*diag_matmul_left(diag(A2D2@A),A) + 3*A2@(A2*A)
        + 3*(A*A2)@A2 + diag_matmul_right(A,diag(A@D3A)) + diag_matmul_left(diag(A@D3A),A) + diag_matmul_left(D2,AD3)
        + diag_matmul_right(D3A,D2) + 3*A@(A*A3) + 3*(A*A3)@A + diag(A@D4@A)
        + 4*D2*D4 + 2*diag(A@diag_matmul_left(D2,A3)) + diag(A2D2@A2)
        + 3*A*A4 + 3*A@(A*A2*A2) + 3*(A*A2*A2)@A
        + 2*diag(AD3@A2) + 2*(D3*D3) + 6*A*A2*A3
        + A2*A2*A2
    )-(
        10*AD2A + 2*diag(A@diag_matmul_left(D22,A)) + 6*D22*D2
        + 7*diag_matmul_left(D2,A2) + 7*A2D2 + diag(A@diag_matmul_left(diag(AD2A),A))
        + 4*D2*diag(AD2A) + 13*AD3 + 13*D3A
        + 3*A*(AD2A) + 10*D4 + 4*A2*A2 + 4*A@(A*A2)
        + 4*(A*A2)@A + 12*diag_matmul_right((A*A2),D2) + 12*diag_matmul_left(D2,(A*A2))
        + 12*diag(torch.matrix_power((A*A2), 2)) + 3*diag(A@(A*A2)@A) + 8*A*(A@(A*A2))
        + 8*A*((A*A2)@A)
    )+(
        8*A2 + 2*diag(AD2A) + 18*D22 + 8*diag(AD2A)
        + 44*A*A2 + 4*A2 + 8*D3
    )- 12*D2)

    del A2, A3, A4, A5, A6
    del D2, D3, D4, D5, D6
    del AD2A, D22, A2D2, D3A, AD3
    
    return P6

# Implement the computational expression of P(7) proposed in this paper, where the detailed expression is contained in P(6) and P(7) Equation.pdf in the supplementary materials.

In [7]:
import time
def compute_P7(A):
    A2 = torch.matrix_power(A, 2)
    A3 = torch.matrix_power(A, 3)
    A4 = torch.matrix_power(A, 4)
    A5 = torch.matrix_power(A, 5)
    A6 = torch.matrix_power(A, 6)
    A7 = torch.matrix_power(A, 7)

    D2 = diag(A2)
    D3 = diag(A3)
    D4 = diag(A4)
    D5 = diag(A5)
    D6 = diag(A6)
    D7 = diag(A7)
    A2D2 = diag_matmul_right(A2,D2)
    D3A = diag_matmul_left(D3,A)
    AD3 = diag_matmul_right(A,D3)
    D2A  = diag_matmul_left(D2,A)
    AD2A = A@D2A
    D2A2 = diag_matmul_left(D2,A2)
    AD2 = diag_matmul_right(A,D2)
    A2D2A = A2D2@A
    dAD2Ad_A = diag_matmul_left(diag(AD2A),A)
    AD3A = AD3@A
    D3A2 = diag_matmul_left(D3,A2)
    D4A = diag_matmul_left(D4,A)
    AD4 = diag_matmul_right(A,D4)
    bA_A2bA = (A*A2)@A
    AD2A2 = AD2@A2
    AbA_A2b = A@(A*A2)
    D2D2A = diag_matmul_left((D2*D2),A)
    start = time.time()
    B = common_neighbor_similarity_sparse_cpu(A)
    end = time.time()
    CNR_time = end - start
    print(f"CNR 耗时: {end - start:.6f} 秒")
    P7 = (36*A) - (
        (66*AD2) + (66*D2A) + (144*D3) + (112*A2*A)
    ) + (
        (21*A3) + (18*diag_matmul_left(D2*D2,A)) + (18*diag_matmul_right(A,D2*D2)) 
        + (14*dAD2Ad_A) + (14*A@diag(AD2A)) + (56*bA_A2bA) + (56*AbA_A2b) 
        + (18*(D2*D3)) + (8*diag(AD3A)) + (12*diag(A2D2A)) + (47*A*A3) + (4*diag_matmul_right(D2A,D2))
        + (18*D2D2A) + (18*diag_matmul_right(A,(D2*D2))) 
        + (76*(D2*D3)) + (32*diag(AD2A2)) + (9*diag(AD3A)) + (8*diag(A2D2A)) 
        + (16*D2A@D2) + (88*A2*A*A2) + (16*A*(AD2A)) + (8*D3A) + (8*AD3) 
        + (36*A*(bA_A2bA)) + (36*A*(AbA_A2b)) + (16*diag_matmul_left(D2,(A2*A))) + (16*diag_matmul_right((A2*A),D2)) 
        + (20*diag(A@bA_A2bA)) + (32*diag((A2*A)@A2)) + (5*(A*A3)) 
        + (11*A*A2*A2*B) + (11*A2*A2*B)
    ) - (
        (11*AD2A2) + (11*A2D2A) + (8*D2@A3) + (8*A3@D2) + (17*AD3A) + (14*D3A2) 
        + (14*A2@D3) + (14*D4A) + (14*AD4) + (11*D5) 
        + (6*diag_matmul_left((D2*D2*D2),A)) + (6*diag_matmul_right(A,(D2*D2*D2))) 
        + (2*diag(A@D2D2A)@A) + (2*A@diag(A@D2D2A)) 
        + (12*(A*A2)@D2A) + (12*AD2@(A*A2)) + (18*(D2*D2)*D3) + (4*diag(AD3@D2A)) 
        + (4*diag(AD2@D2A2)) + (12*diag_matmul_left(D2,bA_A2bA)) + (12*diag_matmul_right(AbA_A2b,D2)) 
        + (4*diag_matmul_left(D2,dAD2Ad_A)) + (4*AD2@diag(AD2A)) + (12*diag_matmul_left(D2,(A3*A))) + (12*diag_matmul_right((A3*A),D2)) 
        + (2*diag_matmul_left((D2*D2),AD2)) + (2*diag_matmul_right(D2A,(D2*D2))) 
        + (4*A@bA_A2bA) + (4*(A*A2)@A2) + (4*A2@(A*A2)) + (diag(A@dAD2Ad_A)@A) 
        + diag_matmul_right(A,diag(A@dAD2Ad_A)) + (4*(A2*A2)@A) + (4*A@(A2*A2)) 
        + (4*D2*diag(AD3A)) + (diag(A@diag(AD3A)@A)) + (4*diag(AD2A)*D3) 
        + (2*diag(A@diag_matmul_left(diag(AD2A),A2))) + (4*A3*A2) + (diag_matmul_right(D2A,diag(AD2A))) + (diag_matmul_left(diag(AD2A),AD2)) 
        + (12*diag_matmul_left(diag(torch.matrix_power(A2*A,2)),A)) + (12*diag_matmul_right(A,diag(torch.matrix_power(A2*A,2)))) 
        + (12*diag_matmul_left(D3,(A2*A))) + (12*diag_matmul_right((A2*A),D3)) + (12*diag_matmul_left(D2,(A2*A*A2))) 
        + (12*diag_matmul_right((A2*A*A2),D2)) + (3*diag_matmul_left(diag(A@bA_A2bA),A)) + (3*A@diag(A@bA_A2bA)) + (8*(A*(bA_A2bA))@A) 
        + (8*A@(A*(AbA_A2b))) + (3*diag(A@(A*A3)@A)) 
        + (24*diag((A*A2)@A3)) + (6*diag(A2@(A2*A)@A)) + (8*(A*(AbA_A2b))@A) + (8*A@(A*(bA_A2bA))) 
        + (3*((AD2A)*A)@A) + (3*A@((AD2A)*A)) 
        + (8*A*(A@(A*A3))) + (8*A*((A*A3)@A)) + (3*(A*A2)@AD2) + (3*D2A@(A*A2)) + (8*A2*(AbA_A2b)) 
        + (8*A2*(bA_A2bA)) + (4*A*(A@(A2*A2))) + (4*A*((A2*A2)@A)) 
        + (8*D2*diag(AD2A2)) + (2*diag(A@diag_matmul_left(diag(AD2A2),A))) + (diag(AD2A@D2A)) + (6*A*A2*(AD2A)) 
        + (3*A*(AD2A2)) + (3*A*(A2D2A)) + (3*A*(AD3A)) 
        + (8*diag(A2@(A*A2*A2))) + (3*diag(A@(A*A2*A2)@A)) + (16*A*((A*A2)@(A*A2)))
    ) + (
        (5*A5) + (2*A@diag_matmul_left((D2*D2),A2)) + (2*A2@D2D2A) 
        + (2*diag_matmul_left((D2*D2),A3)) + (2*diag_matmul_right(A3,(D2*D2))) 
        + (diag_matmul_left(diag(AD2A),A3)) + (A3@diag(AD2A)) + (A@diag(AD2A)@A2) + (A2@dAD2Ad_A) 
        + (4*AD2@D3A) + (4*diag_matmul_left(D2,D3A2)) + (4*diag_matmul_right(A2,(D2*D3))) + (4*diag_matmul_left(D2,D4A)) 
        + (4*diag_matmul_right(AD2,D4)) + (3*AbA_A2b@A2) + (3*A2@bA_A2bA) + (3*(A*A2)@A3) + (3*A3@(A*A2)) 
        + (A@diag(AD3A)@A) + (3*A@(A*A3)@A) + (diag_matmul_left(diag(AD3A),A2)) 
        + (A2@diag(AD3A)) + (3*(A*A3)@A2) + (3*A2@(A*A3)) + (diag_matmul_left(diag(AD4@A),A)) + (A@diag(AD4@A)) 
        + (3*(A*A4)@A) + (3*A@(A*A4)) + (2*A@diag(A2D2A)@A) 
        + (2*diag_matmul_left(diag(AD2A2),A2)) + (2*A2@diag(AD2A2)) + (AD2A@D2A) + (D2A@D2A2) + (diag_matmul_right(A2D2A,D2)) 
        + (2*diag(AD2@A3)@A) + (2*A@diag(AD2@A3)) + (diag_matmul_left(D3,AD2A)) 
        + (diag_matmul_right(AD2A,D3)) + (diag_matmul_left(diag(A2@D2A2),A)) + (A@diag(A2@D2A2)) + (4*(D2*D5)) + (diag(diag_matmul_right(A,D5)@A)) 
        + (2*diag(A4@D2A)) + (2*diag(A3@D2A2)) + (diag_matmul_right(AD2A2,D2)) + (D2A2@D2A) 
        + (3*A*A5) + (D2A@D3A) + (AD3@AD2) + (diag_matmul_right(D2A,D4)) + (diag_matmul_left(D4,AD2)) + (diag_matmul_right(D2A2,D3)) + (diag_matmul_right(D3A2,D2)) 
        + (diag_matmul_left(D2,A3@D2)) + (2*diag_matmul_left((D3*D3),A)) 
        + (2*diag_matmul_right(A,(D3*D3))) + (3*A@(A2*A*A2)@A) + (3*(A2*A*A2)@A2) + (3*A2@(A2*A*A2)) 
        + (2*diag_matmul_left(diag(AD3@A2),A)) + (2*A@diag(AD3@A2)) + (6*(A3*A*A2)@A) 
        + (6*A@(A3*A*A2)) + (4*(D3*D4)) + (2*diag(A2@D4A)) + (2*diag(AD3@A3)) + (diag(A2@D3A2)) 
        + (A@(A2*A2*A2)) + ((A2*A2*A2)@A) + (6*(A*A2*A4)) + diag_matmul_left(D3,AD3) 
        + (3*(A*A3*A3)) + (3*(A2*A2*A3))
    ) - (
        (A3@D2A2) + (A2@diag_matmul_left(D2,A3)) + (AD2@A4) + (A4@D2A) + (diag_matmul_left(D2,A5)) + (diag_matmul_right(A5,D2)) + (A2@D3A2) + (AD3@A3) 
        + (A3@D3A) + (AD4@A2) + (A2@D4A) + (diag_matmul_right(A,D5)@A) + (diag_matmul_left(D3,A4)) 
        + (diag_matmul_right(A4,D3)) + (diag_matmul_left(D4,A3)) + (diag_matmul_right(A3,D4)) + (diag_matmul_left(D5,A2)) + (diag_matmul_right(A2,D5)) 
        + (diag_matmul_left(D6,A)) + (diag_matmul_right(A,D6)) + (D7)) + A7
    del A2, A3, A4, A5, A6, A7
    del D2, D3, D4, D5, D6, D7
    del A2D2, D3A, AD3, D2A, AD2A, D2A2, AD2, A2D2A, dAD2Ad_A, AD3A, D3A2, D4A, AD4, bA_A2bA, AD2A2, AbA_A2b, D2D2A
    return P7,CNR_time

# Compute the Frobenius norm, as mentioned in the experimental section of this paper. Since it has been verified that the expressions of the Revised LRA proposed in this paper for k=5, as well as the expressions of P(6) and P(7), are correct, the following will serve as the relative errors of LRA and Revised LRA at k=5, 6, and 7: the ratio of the Frobenius norm of the difference between the results of LRA and Revised LRA at k=5 to the Frobenius norm of Revised LRA; and the ratios of the Frobenius norms of the differences between LRA/Revised LRA (at k=6 and k=7) and P(6)/P(7) respectively to the Frobenius norms of P(6) and P(7) respectively.

In [8]:
import torch

def frobenius_ratio(A: torch.Tensor, B: torch.Tensor) -> float:
    """
    Compute the ratio of the Frobenius norm of (A-B) to the Frobenius norm of B
    
    Args:
        A: Input matrix 1 (torch.Tensor)
        B: Input matrix 2 (torch.Tensor)
    
    Returns:
        ratio: (||A-B||_F) / (||B||_F)
    """
    if A.shape != B.shape:
        raise ValueError("A and B must have the same shape.")
    
    # Prevent division by zero
    epsilon = 1e-8  
    
    # Calculate Frobenius norms
    diff_norm = torch.norm(A - B, p='fro')
    B_norm = torch.norm(B, p='fro')
    
    return (diff_norm / (B_norm + epsilon)).item()

# Convert the graph data from SNAP into PyTorch adjacency matrices for subsequent experiments.If the program reports an error, you may need to change the filepath in the code to the absolute path of the CA_GrQc.txt file on your device.

In [ ]:
import torch
import networkx as nx
import numpy as np
import re

def snap_to_pytorch(file_path, is_directed=False):
    """
    Convert SNAP dataset to PyTorch adjacency matrix, supporting non-integer node identifiers
    
    Parameters:
        file_path: Path to the edge list file
        is_directed: Whether the graph is directed
    Returns:
        adj: PyTorch sparse adjacency matrix
        id_mapping: Mapping from original node identifiers to integer indices
    """
    # 1. Read all edges as strings (without forced type conversion)
    GraphClass = nx.DiGraph if is_directed else nx.Graph
    G = GraphClass()
    
    # Custom reading function to skip lines that cannot be parsed as edges
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
           # Skip comments and empty lines
            if not line or line.startswith(('#', '%', '//', '{', '}')):
                continue
            
           # Use regular expressions to extract possible node pairs (supports multiple separators)
            parts = re.split(r'\s+', line)  # Split by any whitespace character
            if len(parts) < 2:
                continue  # At least two elements are needed to form an edge
            
            u, v = parts[0], parts[1]
            # Filter obviously invalid node identifiers (e.g., empty strings, blocks of special symbols)
            if not u or not v or re.fullmatch(r'[^a-zA-Z0-9_-]+', u) or re.fullmatch(r'[^a-zA-Z0-9_-]+', v):
                continue
            
            G.add_edge(u, v)
    
    # Check if there are valid edges
    if len(G.edges()) == 0:
        raise ValueError("未找到有效边数据，请检查文件是否为边列表格式")
    
   # 2. Map all nodes (regardless of original type) to consecutive integers
    nodes = sorted(G.nodes())
    n = len(nodes)
    id_mapping = {node: i for i, node in enumerate(nodes)}
    
    # 3. Convert edges to integer indices
    edges = []
    for u, v in G.edges():
        edges.append((id_mapping[u], id_mapping[v]))
    
    # 4. Construct PyTorch sparse adjacency matrix
    edges_np = np.array(edges, dtype=np.int64).T
    values = torch.ones(edges_np.shape[1], dtype=torch.float32)
    adj = torch.sparse_coo_tensor(
        indices=torch.from_numpy(edges_np),
        values=values,
        size=(n, n),
        dtype=torch.float32
    ).coalesce()
    
    return adj, id_mapping

    
file_path = 'CA-GrQc.txt'
A, id_mapping = snap_to_pytorch(
    file_path, 
    is_directed=False
)
print(f"处理完成：")
print(f"节点数量: {len(id_mapping)}")
print(f"边数量: {A._nnz()}")
print(f"邻接矩阵形状: {A.shape}")
A = A.to_dense()


处理完成：
节点数量: 5242
边数量: 14496
邻接矩阵形状: torch.Size([5242, 5242])


# Experiments conducted on the CA_GrqC dataset, with results obtained from running on the A6000 server of our research group, are shown in the rightmost line graph of Figure 3 in the experimental section of this paper.

In [ ]:
import time
import torch
import torch
import networkx as nx
import matplotlib.pyplot as plt

    
# Ensure GPU operations are synchronized (if using GPU)
if torch.cuda.is_available():
    torch.cuda.synchronize()

# 1. lra_5_hop(A)
start = time.time()
lra_5 = lra_5_hop(A)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()
lra_5_times= end - start
# 2. lra_k_hop(A, 6)
start = time.time()
lra_6 = lra_k_hop(A, 6)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()
lra_6_times=end - start

# 3. lra_k_hop(A, 7)
start = time.time()
lra_7 = lra_k_hop(A, 7)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()
lra_7_times=end - start

# 4. lra_5_hop_revised(A)
start = time.time()
lra_5_revised = lra_5_hop_revised(A)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()
lra_5_revised_times=end - start

# 5. lra_k_hop_revised(A, 6)
start = time.time()
lra_6_revised = lra_k_hop_revised(A, 6)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()
lra_6_revised_times=end - start

# 6. lra_k_hop_revised(A, 7)
start = time.time()
lra_7_revised = lra_k_hop_revised(A, 7)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()
lra_7_revised_times=end - start

# 7. P6(A)
start = time.time()
P6 = compute_P6(A)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()
P6_times=end - start

# 8. P7(A)
start = time.time()
P7,CNR_time = compute_P7(A)
if torch.cuda.is_available():
    torch.cuda.synchronize()
end = time.time()
P7_times=end - start
CNR_times=CNR_time

# Output all time metrics
print(f"lra_5_hop execution time: {lra_5_times:.6f} seconds")
print(f"lra_6_hop execution time: {lra_6_times:.6f} seconds")
print(f"lra_7_hop execution time: {lra_7_times:.6f} seconds")
print(f"lra_5_revised execution time: {lra_5_revised_times:.6f} seconds")
print(f"lra_6_revised execution time: {lra_6_revised_times:.6f} seconds")
print(f"lra_7_revised execution time: {lra_7_revised_times:.6f} seconds")
print(f"P6 computation time: {P6_times:.6f} seconds")
print(f"P7 computation time: {P7_times:.6f} seconds")
print(f"CNR computation time: {CNR_times:.6f} seconds")

# Calculate and record Frobenius norm ratios
frobenius_5_times = frobenius_ratio(lra_5, lra_5_revised)
print(f"Frobenius ratio for 5 times: {frobenius_5_times}")

frobenius_6_times = frobenius_ratio(lra_6, P6)
print(f"Frobenius ratio for 6 times: {frobenius_6_times}")

frobenius_7_times = frobenius_ratio(lra_7, P7)
print(f"Frobenius ratio for 7 times: {frobenius_7_times}")

frobenius_6_revised_times = frobenius_ratio(lra_6_revised, P6)
print(f"Frobenius ratio for 6 revised times: {frobenius_6_revised_times}")

frobenius_7_revised_times = frobenius_ratio(lra_7_revised, P7)
print(f"Frobenius ratio for 7 revised times: {frobenius_7_revised_times}")


del lra_5
del lra_6    
del lra_7    
del lra_5_revised    
del lra_6_revised    
del lra_7_revised    
del P6    
del P7


CNR 耗时: 0.673128 秒
lra_5_hop执行时间: 2.924794 秒
lra_6_hop执行时间: 4.108281 秒
lra_7_hop执行时间: 5.507790 秒
lra_5_revised执行时间: 2.700402 秒
lra_6_revised执行时间: 4.040447 秒
lra_7_revised执行时间: 5.778292 秒
P6计算执行时间: 8.465620 秒
P7计算执行时间: 26.051064 秒
CNR计算耗时: 0.673128 秒
Frobenius ratio for 5 times: 0.002221168950200081
Frobenius ratio for 6 times: 1.0547606945037842
Frobenius ratio for 7 times: 1.1422898769378662
Frobenius ratio for 6 revised times: 1.0547893047332764
Frobenius ratio for 7 revised times: 1.142291784286499
